In [1]:
## classing modelsimport numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn import linear_model


In [99]:
###loading ...

#trainset
hijae = pd.read_csv("hijae_red.csv")

#two test sets
hikari = pd.read_csv("hikari_red.csv").iloc[::,1:]
puree = pd.read_csv("puree_red.csv").iloc[::,1:]


hikari.head()


,winner,side_blue,side_red,game_cnt,win,loss,win_rate,kda,kpg,dpg,apg,gpg,ddpg,dtpg,tpg,cs,newwin,topsin,ggg,kapg
0,True,0,1,171,84,87,2.524492,13.289856,29.129149,30.615931,48.800757,54683.175648,508810.86768,126002.89770,4.207736,578.973080,49.122807,327.908454,2436.021225,77.929906
1,True,0,1,524,340,184,3.112952,15.301384,32.467784,27.365665,48.919751,59173.322780,661648.98620,117292.67598,5.184850,670.586541,64.885496,421.982155,5799.569765,81.387535
2,True,0,1,192,106,86,2.987665,14.876406,37.800754,29.220006,47.722817,60760.547372,659445.19410,130546.56087,5.090191,631.071664,55.208333,435.331326,6370.342511,85.523571
3,False,0,1,127,68,59,2.464835,14.576752,26.320000,27.150916,48.227363,58085.646520,645821.42210,105178.93227,4.944359,785.272711,53.543307,368.588921,2094.474072,74.547363
4,False,0,1,199,106,93,2.742903,12.927348,27.499951,30.502745,47.831243,56505.324900,609298.65520,112985.58092,3.875321,598.798477,53.266332,291.932592,5251.979661,75.331194


In [100]:
##The variable selection has been done in R by forward/backward selection. Will soon post R code on this git as well

##EDITION## Manual selection of variables shows similar result. I can provide more logical explaination to this seleciton

y, X = dmatrices('winner ~ tpg + dpg + win_rate + topsin + kapg', hijae, return_type="dataframe")
#flatten y into an array
y = np.ravel(y.iloc[::,1:])


# examine the coefficients

model = LogisticRegression(C=10000000)
model = model.fit(X,y)
model.score(X,y)

print pd.DataFrame(zip(X.columns, np.transpose(model.coef_), np.exp(np.transpose(model.coef_))))


           0                   1                    2
0  Intercept    [-11.5572386899]  [9.56654262533e-06]
1        tpg     [1.72484784525]      [5.61166712292]
2        dpg   [0.0483676860537]      [1.04955649167]
3   win_rate     [5.46011776924]      [235.125113242]
4     topsin  [-0.0238726284941]     [0.976410068659]
5       kapg    [0.102330151278]      [1.10774913616]


In [101]:
##i'm just setting up matrices for test sets

y2, X2 = dmatrices('winner ~ tpg + dpg + win_rate + topsin + kapg', hikari, return_type="dataframe")
y2 = np.ravel(y2.iloc[::,1:])
print model.score(X2,y2)

y3, X3 = dmatrices('winner ~ tpg + dpg + win_rate + topsin + kapg', puree, return_type="dataframe")
y3 = np.ravel(y3.iloc[::,1:])
model.score(X3,y3)

0.85393258427


0.82758620689655171

In [102]:
##What's the actual win rate for each sets of data?

#training set
print "training set:", y.mean()

#test sets
print "hikari set:", y2.mean()
print "puree set:", y3.mean()

training set: 0.4475
hikari set: 0.52808988764
puree set: 0.525862068966


In [ ]:

def mk_checkdf(model_ob, y_ob, x_ob, percentage):
    to55=[]
    for score in model_ob.predict_proba(x_ob)[:,1]:
        if score >= percentage:
            to55.append(1)
        else:
            to55.append(0)
    
    checkdf=pd.DataFrame()
    checkdf['score']=model.predict_proba(x_ob)[:,1]
    checkdf['prediction']=model.predict(x_ob)
    checkdf['real']=y_ob
    checkdf['to55']=to55
    return checkdf


def accuracy(df, win):
    popo=0
    pone=0
    for i in range(0, len(df)):
        if df['to55'][i] == win:
            if df['real'][i] == win:
                popo=popo+1
            else:
                pone=pone+1
    return float(popo)/float(popo+pone)
    #popo+pone = po
    
checkdf= mk_checkdf(model,y2, X2, 0.6)
#print "hikari win pred", accuracy(checkdf, 1)
#print "hikari loss pred", accuracy(checkdf, 0)

checkdf= mk_checkdf(model,y3, X3, 0.5)
#print "puree win", accuracy(checkdf, 1)
#print "puree loss pred", accuracy(checkdf, 0)
#y2.mean()*accuracy(checkdf, 1)+ (1-y2.mean())*accuracy(checkdf, 0)


#print "accuracy", 
print "mean",y.mean()
checkdf = mk_checkdf(model, y, X, 0.5)
print "hijae win", accuracy(checkdf, 1)
print float(y.mean()*float(accuracy(checkdf, 1))+ (1-y.mean())*float(accuracy(checkdf, 0)))
model.score(X,y)